# Setup

In [25]:
import pandas as pd
from tqdm import tqdm
import re
import json
import nltk
import numpy as np
import matplotlib.pyplot as plt

from random import shuffle

from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import DataCollatorForLanguageModeling, AutoTokenizer, TFAutoModelForCausalLM, AdamWeightDecay
from os.path import join, exists

In [26]:
from lib.BBData import character_dict, model_name
from lib.BBDataLoad import dialogpt_preprocess_function, load_char_df, get_chatbot_predictions, merge_df_for_metrics

from lib.wip.frequency_v2 import sentence_preprocess, freq_pairwise_sim, filter_by_weights, get_word_frequency, get_tfidfs, FrequencyChatbotClassifier
from sklearn.metrics import confusion_matrix

In [27]:
characters = list(character_dict.keys())
# characters.remove('Default')

mass_value = 0.3

In [28]:
# Mount google drive, if in Colaboratory environment
import os
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    base_folder = '/content/drive/My Drive/unibo/NLP_project/BarneyBot'
    os.system("pip install datasets")
    os.system("pip install transformers")
    os.system("pip install rouge_score")
    os.system("pip install -U sentence-transformers")
else:
    # base_folder = os.getcwd()
    base_folder = '..'

out_folder = os.path.join(base_folder, 'Data', 'Characters')

# Data Loading

In [29]:
character_docs = dict()
class_docs = [[], []]
for character in characters:
    if character == 'Default':
        df = pd.read_csv(os.path.join(out_folder, character, f'{character}.tsv'), 
                         names=[character, 'response'], sep='\t')
        df['response'] = df['response'].apply(lambda x: x[3:])
        df[character] = df[character].apply(lambda x: x[3:])
    else:
        df = pd.read_csv(os.path.join(out_folder, character, f'{character}.csv'))
    tmp_list = df['response'].tolist()
    character_docs[character] = tmp_list
    class_docs[0] += tmp_list
    class_docs[1] += [character for _ in tmp_list]

In [30]:
for character in characters:
    for i in tqdm(range(len(character_docs[character]))):
        sentence, relevant = sentence_preprocess(character_docs[character][i])
        if relevant:
            character_docs[character][i] = sentence

100%|██████████| 737332/737332 [05:50<00:00, 2105.64it/s]


In [31]:
test_size = 0.33
character_docs_train = {}
character_docs_test = {}
for c in characters:
    shuffle(character_docs[c])
    end_idx = int(len(character_docs[c]) * test_size)
    character_docs_train[c] = character_docs[c][end_idx:]
    character_docs_test[c] = character_docs[c][:end_idx]

# Word Frequency

Loading word frequencies

In [32]:
"""wordfreqs_train = dict()
for character in tqdm(characters):
    wordfreqs_train[character] = get_word_frequency(' '.join(character_docs_train[character]), f_sorted=True)

wordfreqs_test = dict()
for character in tqdm(characters):
    wordfreqs_test[character] = get_word_frequency(' '.join(character_docs_test[character]), f_sorted=True)"""

"wordfreqs_train = dict()\nfor character in tqdm(characters):\n    wordfreqs_train[character] = get_word_frequency(' '.join(character_docs_train[character]), f_sorted=True)\n\nwordfreqs_test = dict()\nfor character in tqdm(characters):\n    wordfreqs_test[character] = get_word_frequency(' '.join(character_docs_test[character]), f_sorted=True)"

In [33]:
"""wordfreqs_reduced_train = dict()
for character in characters:
    wordfreqs_reduced_train[character] = filter_by_weights(wordfreqs_train[character], mass=0.3)

wordfreqs_reduced_test = dict()
for character in characters:
    wordfreqs_reduced_test[character] = filter_by_weights(wordfreqs_test[character], mass=0.3)"""

'wordfreqs_reduced_train = dict()\nfor character in characters:\n    wordfreqs_reduced_train[character] = filter_by_weights(wordfreqs_train[character], mass=0.3)\n\nwordfreqs_reduced_test = dict()\nfor character in characters:\n    wordfreqs_reduced_test[character] = filter_by_weights(wordfreqs_test[character], mass=0.3)'

Loading TF-IDF matrix

In [34]:
"""tfidf_vectorizer = TfidfVectorizer(input='content', stop_words='english')"""

"tfidf_vectorizer = TfidfVectorizer(input='content', stop_words='english')"

In [35]:
"""tfidfs_train = get_tfidfs([' '.join(character_docs_train[character]) for character in characters], characters, tfidf_vectorizer)"""

"tfidfs_train = get_tfidfs([' '.join(character_docs_train[character]) for character in characters], characters, tfidf_vectorizer)"

In [36]:
"""tfidfs_reduced_train = dict()
for character in characters:
    tfidfs_reduced_train[character] = filter_by_weights(tfidfs_train[character], mass=mass_value)"""

'tfidfs_reduced_train = dict()\nfor character in characters:\n    tfidfs_reduced_train[character] = filter_by_weights(tfidfs_train[character], mass=mass_value)'

# WordCloud Plot (Example)

In [37]:
"""from lib.BBVisualizations import BBVisualization
BBVisualization.load_visualization("wordcloud").plot(freqdict=tfidfs_reduced_train['Barney'])"""

'from lib.BBVisualizations import BBVisualization\nBBVisualization.load_visualization("wordcloud").plot(freqdict=tfidfs_reduced_train[\'Barney\'])'

# Frequency Pairwise Similarity (Example)

In [38]:
"""freq_pairwise_sim(wordfreqs_reduced_train['Fry'], wordfreqs_reduced_test['Fry'])"""

"freq_pairwise_sim(wordfreqs_reduced_train['Fry'], wordfreqs_reduced_test['Fry'])"

# Test Classifier Performances Against Test Set

In [39]:
"""wf_classifier = FrequencyChatbotClassifier(characters, mode='word frequency')
wf_classifier.train(list(character_docs_train.values()))

y_true = range(0, len(characters))
y_pred = [np.argmax(list(wf_classifier.predict(character_docs_test[character], mass=0.3).values()))
          for character in characters]

print('Word Frequency classifier test accuracy: {:.2f}'.format(
             sum([y_pred[i] == y_true[i] for i in range(len(y_true))]) / len(y_true)))
confusion_matrix(y_true, y_pred)"""

"wf_classifier = FrequencyChatbotClassifier(characters, mode='word frequency')\nwf_classifier.train(list(character_docs_train.values()))\n\ny_true = range(0, len(characters))\ny_pred = [np.argmax(list(wf_classifier.predict(character_docs_test[character], mass=0.3).values()))\n          for character in characters]\n\nprint('Word Frequency classifier test accuracy: {:.2f}'.format(\n             sum([y_pred[i] == y_true[i] for i in range(len(y_true))]) / len(y_true)))\nconfusion_matrix(y_true, y_pred)"

In [40]:
tfidf_classifier = FrequencyChatbotClassifier(characters, mode='tf-idf')
tfidf_classifier.train(list(character_docs_train.values()))

y_true = range(0, len(characters))
y_pred = []
raw_predictions = {c:[] for c in characters}
msw_predictions = {c:[] for c in characters}
for c in tqdm(range(len(characters))):
    prediction = tfidf_classifier.predict(character_docs_test[characters[c]], mass=0.3)
    msws = tfidf_classifier.get_MSW(character_docs_test[characters[c]], mass=0.3)
    raw_predictions[characters[c]].append(prediction)
    msw_predictions[characters[c]].append(msws)
    y_pred.append(np.argmax(list(prediction.values())))

print('TF-IDF classifier test accuracy: {:.2f}'.format(
             sum([y_pred[i] == y_true[i] for i in range(len(y_true))]) / len(y_true)))
confusion_matrix(y_true, y_pred)

100%|██████████| 9/9 [04:40<00:00, 31.17s/it]

TF-IDF classifier test accuracy: 1.00


array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int64)

In [41]:
# raw_predictions

In [42]:
# msw_predictions

### c-TF-IDF 

In [43]:
ctfidf_classifier = FrequencyChatbotClassifier(characters, mode='c-tf-idf')
ctfidf_classifier.train(list(character_docs_train.values()))

y_true = range(0, len(characters))
y_pred = []
raw_predictions = {c:[] for c in characters}
# msw_predictions = {c:[] for c in characters}
for c in tqdm(range(len(characters))):
    prediction = ctfidf_classifier.predict(character_docs_test[characters[c]], mass=0.3)
    # msws = ctfidf_classifier.get_MSW(character_docs_test[characters[c]], mass=0.3)
    raw_predictions[characters[c]].append(prediction)
    # msw_predictions[characters[c]].append(msws)
    # print(prediction)
    y_pred.append(np.argmax(prediction))

print('TF-IDF classifier test accuracy: {:.2f}'.format(
             sum([y_pred[i] == y_true[i] for i in range(len(y_true))]) / len(y_true)))
confusion_matrix(y_true, y_pred)

100%|██████████| 9/9 [00:04<00:00,  2.10it/s]

TF-IDF classifier test accuracy: 1.00


array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int64)

In [44]:
raw_predictions

{'Barney': [array([0.99617002, 0.95629725, 0.94193125, 0.98012821, 0.97783672,
         0.88733131, 0.96534351, 0.95859173, 0.9230213 ])],
 'Sheldon': [array([0.95906242, 0.99856711, 0.88759811, 0.93746857, 0.94623484,
         0.89946734, 0.92078986, 0.91341358, 0.94372555])],
 'Harry': [array([0.95425066, 0.8984451 , 0.96260125, 0.95624752, 0.94756437,
         0.82543339, 0.93725922, 0.92891611, 0.86317837])],
 'Fry': [array([0.97656237, 0.92699509, 0.95475201, 0.99176699, 0.98475479,
         0.84249239, 0.96418706, 0.95417869, 0.90986188])],
 'Bender': [array([0.97701121, 0.94257317, 0.94570096, 0.98707818, 0.99232888,
         0.85880576, 0.96061848, 0.94764694, 0.92220425])],
 'Vader': [array([0.87145794, 0.89458761, 0.81617367, 0.84789941, 0.85536314,
         0.94279504, 0.78275262, 0.7699793 , 0.91107325])],
 'Joey': [array([0.96410748, 0.92616945, 0.92820502, 0.967309  , 0.96183399,
         0.82336805, 0.99708326, 0.9850087 , 0.87827135])],
 'Phoebe': [array([0.94824368, 0.

# Test Classifiers On Chatbots (Not Cached!)

In [45]:
tfidf_classifier = FrequencyChatbotClassifier(characters, mode='c-tf-idf')
tfidf_classifier.train(list(character_docs.values()))

In [46]:
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=join("..", "cache"))
tokenizer.pad_token = '#'
data_collator = DataCollatorForLanguageModeling(mlm=False, tokenizer=tokenizer, return_tensors='tf')

In [49]:
characters_noDefault = characters.copy()
characters_noDefault.remove('Default')

In [50]:
### create dataset
n_tests = 1
doc_test = []
batch_size = 8
override_predictions = False
predictions = {c:[] for c in characters}
raw_predictions = {c:[] for c in characters}
msw_predictions = {c:[] for c in characters}
print('Creating dataset...')
if n_tests > 1 and not override_predictions:
    raise Exception('must override previous predictions if you need more tests')

for i in range(n_tests):
    print(f'Run {i}/{n_tests}')
    for character in tqdm(characters_noDefault):
        character_checkpoint = join(out_folder, character, character_dict[character]['checkpoint_folder'])
        model_chatbot = TFAutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=character_checkpoint) if override_predictions else None
        if model_chatbot:
            model_chatbot.compile(optimizer=AdamWeightDecay(learning_rate=2e-5))

        character_hg = load_char_df(character, base_folder)
        # This transform in a sequence of tokens ours dataset
        tokenized_character_hg = character_hg.map(lambda row: dialogpt_preprocess_function(row, tokenizer), batched=False)

        # Define tensorflow datasets
        encoded_test_set = tokenized_character_hg["test"].to_tf_dataset(
            columns=["input_ids", "attention_mask", "labels"],
            shuffle=False,
            batch_size=batch_size,
            collate_fn=data_collator,
        )

        # Takes the testset as sample question 
        sample_questions = character_hg['test']['context/0']

        # Sampling generation method
        predictions_sampling = get_chatbot_predictions(
            sample_questions,
            model_chatbot,
            character_dict[character]['prediction_filename'] + '_sampling.json',
            "Sampling",
            character,
            tokenizer,
            base_folder,
            override_predictions=override_predictions
        )
                                                    
        sentences = merge_df_for_metrics(character_hg['test'], None, None, predictions_sampling, tokenizer)['prd_sampling'].tolist()
        sentences = [sentence_preprocess(s) for s in sentences]
        doc_test.append([s[0] for s in sentences if s[1]])

Creating dataset...
Run 0/1


  0%|          | 0/8 [00:00<?, ?it/s]Using custom data configuration default-d78476db32c8db98
Reusing dataset csv (cache\csv\default-d78476db32c8db98\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
100%|██████████| 1/1 [00:00<00:00, 126.10it/s]
Loading cached split indices for dataset at cache\csv\default-d78476db32c8db98\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-651037d43e5e3dbd.arrow and cache\csv\default-d78476db32c8db98\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-7e4903d85e508b43.arrow
Loading cached split indices for dataset at cache\csv\default-d78476db32c8db98\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-1b018c4ce0c1ed8c.arrow and cache\csv\default-d78476db32c8db98\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-2cfa4b0ef22c9ad5.arrow
Loading cached processed dataset at cache\csv\default-d78476db32c8db98\0.0.0\433e0ccc46f9880962

Loading predictions from stored file
Loaded predictions from stored file


 12%|█▎        | 1/8 [00:02<00:19,  2.72s/it]Using custom data configuration default-8207271dc09e0a46
Reusing dataset csv (cache\csv\default-8207271dc09e0a46\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
100%|██████████| 1/1 [00:00<00:00, 122.50it/s]
Loading cached split indices for dataset at cache\csv\default-8207271dc09e0a46\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-46b6cd7566c1d243.arrow and cache\csv\default-8207271dc09e0a46\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-e3e40e36f25dded7.arrow
Loading cached split indices for dataset at cache\csv\default-8207271dc09e0a46\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-1774625d7360210c.arrow and cache\csv\default-8207271dc09e0a46\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-ad5b734370182aae.arrow
Loading cached processed dataset at cache\csv\default-8207271dc09e0a46\0.0.0\433e0ccc46

Loading predictions from stored file
Loaded predictions from stored file


...

 75%|███████▌  | 6/8 [00:14<00:04,  2.07s/it]Using custom data configuration default-b81141adc87b87bf
Reusing dataset csv (cache\csv\default-b81141adc87b87bf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
100%|██████████| 1/1 [00:00<00:00, 123.68it/s]
Loading cached split indices for dataset at cache\csv\default-b81141adc87b87bf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-e2d3926d0ffd0fd3.arrow and cache\csv\default-b81141adc87b87bf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-f888d4a055d9f5e2.arrow
Loading cached split indices for dataset at cache\csv\default-b81141adc87b87bf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-682a04a266efce80.arrow and cache\csv\default-b81141adc87b87bf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-13fca5dad2d5247a.arrow
Loading cached processed dataset at cache\csv\default-b81141adc87b87bf\0.0.0\433e0ccc46

Loading predictions from stored file
Loaded predictions from stored file


 88%|████████▊ | 7/8 [00:20<00:03,  3.22s/it]Using custom data configuration default-48cc5f719001fd3a
Reusing dataset csv (cache\csv\default-48cc5f719001fd3a\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
100%|██████████| 1/1 [00:00<00:00, 124.20it/s]
Loading cached split indices for dataset at cache\csv\default-48cc5f719001fd3a\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-16c140b5da191e7c.arrow and cache\csv\default-48cc5f719001fd3a\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-ed121df269d87298.arrow
Loading cached split indices for dataset at cache\csv\default-48cc5f719001fd3a\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-f4f26cbce3ecb27c.arrow and cache\csv\default-48cc5f719001fd3a\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-186da54849d8ccad.arrow
Loading cached processed dataset at cache\csv\default-48cc5f719001fd3a\0.0.0\433e0ccc46

Loading predictions from stored file
Loaded predictions from stored file


100%|██████████| 8/8 [00:24<00:00,  3.01s/it]


In [ ]:
### prediction on last test
print('Classification...')
y_pred = []
# raw_predictions = []
for c in tqdm(range(len(characters_noDefault))):
    prediction = tfidf_classifier.predict(character_docs_test[characters[c]], mass=0.3)
    # msws = tfidf_classifier.get_MSW(character_docs_test[characters[c]], mass=0.3)    
    raw_predictions[characters[c]].append(prediction)
    # msw_predictions[characters[c]].append(msws)
    # predictions[characters[c]].append(
    #     int(max(prediction, key=prediction.get) == characters[c])
    # )
    y_pred.append(np.argmax(prediction))

Classification...


100%|██████████| 8/8 [00:00<00:00, 23.19it/s]


In [51]:
raw_predictions

{'Barney': [],
 'Sheldon': [],
 'Harry': [],
 'Fry': [],
 'Bender': [],
 'Vader': [],
 'Joey': [],
 'Phoebe': [],
 'Default': []}

In [52]:
# print('TF-IDF classifier test accuracy: {:.2f}'.format(sum([char_pred[-1] for char_pred in predictions.values()])/len(predictions)))
print('TF-IDF classifier test accuracy: {:.2f}'.format(
             sum([y_pred[i] == y_true[i] for i in range(len(y_true))]) / len(y_true)))
confusion_matrix(y_true, y_pred)

TF-IDF classifier test accuracy: 1.00


array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int64)

In [53]:
### save predictions
append_predictions = True
override_predictions = False
predictions_file = join('..', 'metrics', 'tfidf_predictions.json')

if append_predictions and exists(predictions_file):
    with open(predictions_file, 'r', encoding='utf-8') as file:
        predictions_dict = json.load(file)
elif override_predictions or not exists(predictions_file):
    predictions_dict = {'one_hot':{c:[] for c in characters}, 'raw_predictions': {c:[] for c in characters}}
else:
    raise NotImplementedError
    
for c in characters_noDefault:
    predictions_dict['one_hot'][c] += predictions[c]
    predictions_dict['raw_predictions'][c] += raw_predictions[c]

with open(predictions_file, 'w', encoding='utf-8') as file:
    json.dump(predictions_dict, file)

In [54]:
predictions_dict

{'one_hot': {'Barney': [1, 1, 1, 1, 1],
  'Sheldon': [1, 1, 1, 1, 1],
  'Harry': [1, 0, 0, 0, 1],
  'Fry': [0, 0, 0, 0, 1],
  'Bender': [0, 1, 1, 1, 1],
  'Vader': [1, 0, 0, 0, 1],
  'Joey': [0, 0, 0, 0, 1],
  'Phoebe': [1, 1, 1, 1, 1]},
 'raw_predictions': {'Barney': [{'Barney': 0.7427531645309138,
    'Sheldon': 0.3409307888725184,
    'Harry': 0.24635663538513103,
    'Fry': 0.4365837899414391,
    'Bender': 0.4690792107496431,
    'Vader': 0.016606825033575093,
    'Joey': 0.5216851304420018,
    'Phoebe': 0.5625899764181352},
   {'Barney': 0.78300635943899,
    'Sheldon': 0.3970594438761256,
    'Harry': 0.30613045222591173,
    'Fry': 0.47947848053595993,
    'Bender': 0.5279603147782952,
    'Vader': 0.024668276074521298,
    'Joey': 0.5775909062067571,
    'Phoebe': 0.6605154960250631,
    'Default': 0.46163274137594223},
   {'Barney': 0.78300635943899,
    'Sheldon': 0.3970594438761256,
    'Harry': 0.30613045222591173,
    'Fry': 0.47947848053595993,
    'Bender': 0.527960314